In [2]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

from datetime import date
import pandas as pd
date_today = date.today()

SCROLL_PAUSE_TIME = 5

URL = 'https://thehundreds.com/collections/apparel'
driver = webdriver.Chrome('/Users/mandili/.wdm/drivers/chromedriver/87.0.4280.20/mac64/chromedriver')
driver.get(URL)

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(driver.page_source,'html.parser')

In [38]:
product_url = []
picture_url = []
name = []

timestamp = []
brand = []
product_type = []

price = []
sales_price = []

for i in soup.find_all('div',{'class':'ProductTile AnimateOnHover Animate Animate--slide-up Animate--animated'}):
    
    product_url.append("https://thehundreds.com" + i.find('a',{'class':'ProductTile__image'}).get('href'))
    picture_url.append("https:" + i.find('a',{'class':'ProductTile__image'}).img.get('src'))
    name.append(i.find('h6',{'class':'ProductTile__title'}).a.text)
    
    timestamp.append(date_today)
    brand.append("The Hundreds")
    product_type.append(None)
    
    sales_price.append(i.find('p',{'class':'ProductTile__price'}).span.text)
    price.append(None)


# Second Layer

In [48]:
product_description = []
material = []
picture_name = []
color = []
counter = 1

for link in product_url:
    item_color = []
    item_material = []
    ourUrl = urllib.request.urlopen(link)
    soup = BeautifulSoup(ourUrl,'html.parser')
    
    picture_name.append('the-hundreds_' + str(counter) + '.jpg') # change variable
    counter += 1
    
    item_color = []
    for color_i in soup.find('ul',{'class':'ProductOption__variants ProductOption__variants--list'}).find_all('li'):
        item_color.append(color_i.get('data-value'))
    color.append(item_color)

    material.append(None)
    product_description.append(soup.find('div',{'class':'rte'}).text.replace('• ','').replace('Product Detail','').replace('\n','').replace('Show More','').strip())

In [49]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", "picture_name"]
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = name
second_pass.product_description = product_description
second_pass.price = price
second_pass.sales_price = sales_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = picture_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass.head()

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,The Hundreds,None,Wild Pullover Hoodie,The Hundreds x UNO graphic printed on front an...,None,$95.00,"[Black, Burgundy]",None,https://thehundreds.com/collections/apparel/pr...,https://cdn.shopify.com/s/files/1/1520/5960/pr...,2020-12-08,the-hundreds_1.jpg
1,The Hundreds,None,Adam Bomb Card T-Shirt,The Hundreds x UNO graphic printed on front an...,None,$38.00,"[Cyan, Black, Magenta, Yellow]",None,https://thehundreds.com/collections/apparel/pr...,https://cdn.shopify.com/s/files/1/1520/5960/pr...,2020-12-08,the-hundreds_2.jpg
2,The Hundreds,None,Clicker Jacket,16oz Cotton CanvasDiamond quilting w/ 1600gsm ...,None,$159.00,"[Duck Brown, Black]",None,https://thehundreds.com/collections/apparel/pr...,https://cdn.shopify.com/s/files/1/1520/5960/pr...,2020-12-08,the-hundreds_3.jpg
3,The Hundreds,None,Yosemite Anorak,Nylon w/ milky white backing Water resistant c...,None,$139.00,"[Green, Black, Red]",None,https://thehundreds.com/collections/apparel/pr...,https://cdn.shopify.com/s/files/1/1520/5960/pr...,2020-12-08,the-hundreds_4.jpg
4,The Hundreds,None,Regent Button-Up,Shirt jacket fit 70% Polyester 30% Wool brushe...,None,$129.00,"[Gold, Purple, Red]",None,https://thehundreds.com/collections/apparel/pr...,https://cdn.shopify.com/s/files/1/1520/5960/pr...,2020-12-08,the-hundreds_5.jpg


In [54]:
second_pass.to_csv("the-hundreds.csv", index=False, encoding = 'utf-8-sig')

# Download Images

In [58]:
import os
import requests as req

#new_pd = pd.read_csv('/Users/mandili/Desktop/the-hundreds.csv', encoding='utf-8-sig', error_bad_lines=False)

os.mkdir('the-hundreds')
i = 1
for index, link in enumerate(picture_url):
    img_data = req.get(link).content
    with open('the-hundreds/'+ "the-hundreds_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1